Варіант 1
Завдання до лабораторної роботи
Створити програму, яка:
1. Зчитує Файл twitter1.csv, виконує попередню обробку, розбиває дані
на навчальні на тестові. Виконує аналіз настроїв за допомогою логістичної регресії.
2. Розрахувати матрицю невідповідностей, провести оцінку точності
моделі.
3. Використати один з готових лексиконів, наприклад Textblob, для аналізу
оцінки настроїв. Також розрахувати матрицю невідповідностей,
провести оцінку точності моделі.
4. Обрати три випадкові записи та вивести результати оцінки їх настрою за
пунктами 1 і 3.

In [10]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from textblob import TextBlob

In [11]:
# здійснюємо імпорт таблиці
data = pd.read_csv("twitter1.csv", header=None)
column_names = data.columns.tolist()

print(data.head(5))

      0            1         2   
0  2401  Borderlands  Positive  \
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                   3  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  


In [12]:
# за допомогою функції з поперідньої лабораторної обробляємо наш текст
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def preproc_doc(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc)
    doc = doc.lower().strip()
    tokens = word_tokenize(doc)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

data[3] = data[3].apply(lambda x: preproc_doc(x) if isinstance(x, str) else x)
    
print(data.head(5))

# виконуємо пошук пустих документі, які могли виникнути під час обробки
data = data.replace(r'^(\s?)+$', np.nan, regex=True)
data.info()

# видаляємо ці документи
data = data.dropna().reset_index(drop=True)
data.info()

# зберігаємо три випадкові записи з таблиці в змінну для четвертого завдання
random_samples = data.sample(n=3)
print(random_samples)

      0            1         2                              3
0  2401  Borderlands  Positive  im getting borderlands murder
1  2401  Borderlands  Positive            coming borders kill
2  2401  Borderlands  Positive    im getting borderlands kill
3  2401  Borderlands  Positive   im coming borderlands murder
4  2401  Borderlands  Positive  im getting borderlands murder
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       74682 non-null  int64 
 1   1       74682 non-null  object
 2   2       74682 non-null  object
 3   3       72364 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72364 entries, 0 to 72363
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       72364 non-null  int64 
 1   1       72364 non-null  object
 2

In [13]:
# розділяємо дані на тестову та навчальну вибірки
train_mood, test_mood, train_content, test_content = train_test_split(
    np.array(data[2]),
    np.array(data[3]),
    test_size=0.2,
    random_state=42
)
# створюємо сумку слів
cv = CountVectorizer(binary=False, min_df=0.0, max_df=1.0)
cv_train_content = cv.fit_transform(train_content)
cv_test_content = cv.transform(test_content)
cv_train_content.shape, cv_test_content.shape

((57891, 35750), (14473, 35750))

In [14]:
# ініціалізуємо логістичну регресію
logreg = LogisticRegression(penalty='l2', max_iter=1000, C=1, random_state=0)

# навчаємо модель на навчальних даних
logreg.fit(cv_train_content, train_mood)

# прогнозуємо настрій для тестових даних
pred_mood = logreg.predict(cv_test_content)

# оцінюємо точність моделі
accuracy = accuracy_score(test_mood, pred_mood)
print("Accuracy:", accuracy)

# розрахуємо матрицю невідповідностей
confusion_mtx = confusion_matrix(test_mood, pred_mood)
print("Confusion Matrix:")
print(confusion_mtx)

Accuracy: 0.8408761141435777
Confusion Matrix:
[[1904  192  142  246]
 [  93 3903  191  272]
 [ 112  232 2837  284]
 [ 100  258  181 3526]]


In [15]:
# аналіз настрою коментарів
data[3] = data[3].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# конвертація значень настрою в рядки
data[3] = data[3].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')

# порівняння результатів
accuracy = (data[3] == data[2]).mean()
print("Accuracy:", accuracy)

Accuracy: 0.3995771378033276


In [16]:
# побудова матриці невідповідностей
confusion_mat = confusion_matrix(data[2], data[3])
print("Матриця:")
print(confusion_mat)

Матриця:
[[    0  3266  3334  6030]
 [    0 10386  5731  5798]
 [    0  4330  5393  7926]
 [    0  3167  3867 13136]]


In [17]:
# розділяємо дані на тестову та навчальну вибірки
train_mood, test_mood, train_content, test_content = train_test_split(
    np.array(random_samples[2]),
    np.array(random_samples[3]),
    test_size=0.3,
    random_state=42
)
# створюємо сумку слів
cv = CountVectorizer(binary=False, min_df=0.0, max_df=1.0)
cv_train_content = cv.fit_transform(train_content)
cv_test_content = cv.transform(test_content)
cv_train_content.shape, cv_test_content.shape
# навчаємо моделі на навчальних даних
logreg.fit(cv_train_content, train_mood)

# прогнозуємо настрій для тестових даних
pred_mood = logreg.predict(cv_test_content)

# оцінка точності моделі
accuracy = accuracy_score(test_mood, pred_mood)
print("Accuracy:", accuracy)

# розрахунок матриці невідповідностей
confusion_mtx = confusion_matrix(test_mood, pred_mood)
print("Confusion Matrix:")
print(confusion_mtx)

Accuracy: 0.0
Confusion Matrix:
[[0 0]
 [1 0]]


In [18]:
# аналіз настрою коментарів
random_samples[3] = random_samples[3].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# конвертація значень настрою в рядки
random_samples[3] = random_samples[3].apply(lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral')
print(random_samples.head())
# порівняння результатів
accuracy = (random_samples[3] == random_samples[2]).mean()
print("Accuracy:", accuracy)

# побудова матриці невідповідностей
confusion_mat = confusion_matrix(random_samples[2], random_samples[3])
print("Матриця:")
print(confusion_mat)

           0                     1         2         3
68132  10846  TomClancysGhostRecon   Neutral  Positive
9441   12884         Xbox(Xseries)   Neutral  Positive
14383   2946                 Dota2  Negative  Negative
Accuracy: 0.3333333333333333
Матриця:
[[1 0 0]
 [0 0 2]
 [0 0 0]]
